# Hypothesis Testing 
Part 4

Jacob Tanzi

Using MySQL database from part 3

Answer questions for your stakeholder. 

* *what makes a successful movie??*

#### Questions to Answer

#### Q1
* Does the MPAA rating of a movie (G/PG/PG-13/R) affect how much revenue the movie generates?




#### Q2
Do movies that are over 2 hours long earn more revenue than movies that are 1.5 hours long (or less)?

#### Q3
Do some movie genres earn more revenue than others?


In [7]:
import os, json, math, time
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import tmdbsimple as tmdb 
from tqdm.notebook import tqdm_notebook
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector, ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

* They want you to perform a statistical test to get a mathematically-supported answer.
* They want you to report if you found a significant difference between ratings.
 * If so, what was the p-value of your analysis?
 * And which rating earns the most revenue?
 
* They want you to prepare a visualization that supports your finding.





#### Specifications
#### Your Data

* A critical first step for this assignment will be to retrieve additional movie data to add to your SQL database.
 * You will want to use the TMDB API again and extract data for additional years.
 * You may want to review the optional lesson from Week 1 on "Using Glob to Load Many Files" to load and combine all of your API results for each year.

* However, trying to extract the TMDB data for all movies from 2000-2022 could take >24 hours!

* To address this issue, you should EITHER:
 * Define a smaller (but logical) period of time to use for your analyses (e.g., last 10 years, 2010-2019 (pre-pandemic, etc).

 * OR coordinate with cohort-mates and divide the API calls so that you can all download the data for a smaller number of years and then share your downloaded JSON data.




In [6]:
pd.set_option('display.max_columns',100)

In [8]:
import json
with open('/Users/Jacob/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)

login.keys()

dict_keys(['Client-ID', 'API Key'])

In [9]:
import tmdbsimple as tmdb
tmdb.API_KEY =  login['API Key']

In [10]:
def get_movie_with_rating(movie_id): 
    
    movie = tmdb.Movies(movie_id)

    info = movie.info()
    releases = movie.releases()

    for c in releases['countries']:
   
        if c['iso_3166_1' ] =='US':
       
           info['certification'] = c['certification']
    return info

In [7]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
   
        file_data = json.load(file)
  
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
  
        file.seek(0)

        json.dump(file_data, file)

#### Confirm Your API Function works.


In [11]:
test = get_movie_with_rating("tt0848228")
test

{'adult': False,
 'backdrop_path': '/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 123.731,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path

In [12]:
test = get_movie_with_rating("tt0332280")
test

{'adult': False,
 'backdrop_path': '/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg',
 'belongs_to_collection': None,
 'budget': 29000000,
 'genres': [{'id': 10749, 'name': 'Romance'}, {'id': 18, 'name': 'Drama'}],
 'homepage': 'http://www.newline.com/properties/notebookthe.html',
 'id': 11036,
 'imdb_id': 'tt0332280',
 'original_language': 'en',
 'original_title': 'The Notebook',
 'overview': "An epic love story centered around an older man who reads aloud to a woman with Alzheimer's. From a faded notebook, the old man's words bring to life the story about a couple who is separated by World War II, and is then passionately reunited, seven years later, after they have taken different paths.",
 'popularity': 54.916,
 'poster_path': '/rNzQyW4f8B8cQeg7Dgj3n6eT5k9.jpg',
 'production_companies': [{'id': 12,
   'logo_path': '/5ThIuO93vsk47oexKTSdfKEr7EC.png',
   'name': 'New Line Cinema',
   'origin_country': 'US'},
  {'id': 1565, 'logo_path': None, 'name': 'Avery Pix', 'origin_country': 'US'},
  {'id': 26

In [13]:
errors = [ ]

In [14]:
print(f"- Number of errors: {len(errors)}")
errors

- Number of errors: 0


[]

In [15]:
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'Data',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'TMDB api.ipynb',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'tmdb_results_combined.csv.gz']

In [16]:
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)

new_file_path = os.path.join(FOLDER, "2010-2021")
with open(new_file_path, "w") as new_file:
    new_file.write("This is the content of the new file.")


In [17]:
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 '2010-2021',
 'Data',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'TMDB api.ipynb',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'tmdb_results_combined.csv.gz']

In [21]:
basics = pd.read_csv(r'\Users\Jacob\Documents\GitHub\0424 d e\Project-3\Data\title_basics.csv.gz')

In [22]:
YEARS_TO_GET = list(range(2010, 2022)) 

In [23]:
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    file_exists = os.path.isfile(JSON_FILE)
    
    if file_exists == False:

        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
            
    df = basics.loc[ basics['startYear']==YEAR].copy()
    movie_ids = df['tconst'].copy()
    movie_ids
    
    previous_df = pd.read_json(JSON_FILE)
    previous_df
    
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])
    
    
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)
    
    

YEARS:   0%|          | 0/12 [00:00<?, ?it/s]

Movies from 2010:   0%|          | 0/3864 [00:00<?, ?it/s]

Movies from 2011:   0%|          | 0/4227 [00:00<?, ?it/s]

Movies from 2012:   0%|          | 0/4526 [00:00<?, ?it/s]

Movies from 2013:   0%|          | 0/4713 [00:00<?, ?it/s]

...

Movies from 2017:   0%|          | 0/5643 [00:00<?, ?it/s]

Movies from 2018:   0%|          | 0/5792 [00:00<?, ?it/s]

Movies from 2019:   0%|          | 0/5879 [00:00<?, ?it/s]

Movies from 2020:   0%|          | 0/5014 [00:00<?, ?it/s]

Movies from 2021:   0%|          | 0/5172 [00:00<?, ?it/s]

In [16]:
# YEAR = 2000

#### Deliverables
* You should use the same project repository you have been using for Parts 1-3 (for your portfolio).
 * Create a new notebook in your project repository just for the hypothesis testing (like "Part 4 - Hypothesis Testing.ipynb")
 * Make sure the results and visualization for all 3 hypotheses are in your notebook.


Please submit the link to your GitHub repository for this assignment.
